In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

from miniMTL.datasets import *
from miniMTL.models import *
from miniMTL.util import *
from miniMTL.training import *
from miniMTL.hps import *

In [2]:
# From torch fundamentals course
def train(dataloader, model, loss_fn, name, optimizer,device='cpu'):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y[name].to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# From torch fundamentals course
def test(dataloader, model,loss_fn,name,device='cpu'):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y[name].to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Load data

In [3]:
p_pheno = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/pheno_01-12-21.csv'
p_conn = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/connectomes/'

cases = ['SZ',
        #'BIP',
        #'ASD',
        #'DEL22q11_2',
        #'DEL16p11_2',
        #'DUP16p11_2',
        #'DUP22q11_2',
        #'DEL1q21_1',
        #'DUP1q21_1'
        ]

case = 'SZ'
data = caseControlDataset(case,p_pheno,p_conn,format=2)

/home/harveyaa/miniconda3/envs/MTL/lib/python3.7/site-packages/ipykernel_launcher.py:16: DtypeWarning: Columns (7,8,12,13,14,19,20,24,27,31,42,43,44,49,51,59,60,62,64,65,68,101,121,163) have mixed types.Specify dtype option on import or set low_memory=False.
  app.launch_new_instance()


In [4]:
train_d, test_d = split_data(data)
trainloader = DataLoader(train_d, batch_size=16, shuffle=True)
testloader = DataLoader(test_d, batch_size=16, shuffle=True)

In [5]:
#net = E2EBlock(1,32).double()
net = BrainNetCNN().double()

In [11]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, net, loss_fn, case, optimizer)
    test(testloader, net, loss_fn, case)
print("Done!")

Epoch 1
-------------------------------
loss: 0.704631  [    0/  451]
Test Error: 
 Accuracy: 52.2%, Avg loss: 0.048891 

Epoch 2
-------------------------------
loss: 0.716998  [    0/  451]
Test Error: 
 Accuracy: 48.7%, Avg loss: 0.049000 

Epoch 3
-------------------------------
loss: 0.694933  [    0/  451]
Test Error: 
 Accuracy: 51.3%, Avg loss: 0.049723 

Epoch 4
-------------------------------
loss: 0.624284  [    0/  451]
Test Error: 
 Accuracy: 52.2%, Avg loss: 0.050401 

Epoch 5
-------------------------------
loss: 0.843472  [    0/  451]
Test Error: 
 Accuracy: 55.8%, Avg loss: 0.048735 

Epoch 6
-------------------------------
loss: 0.548061  [    0/  451]
Test Error: 
 Accuracy: 61.9%, Avg loss: 0.043969 

Epoch 7
-------------------------------
loss: 0.518762  [    0/  451]
Test Error: 
 Accuracy: 62.8%, Avg loss: 0.045504 

Epoch 8
-------------------------------
loss: 0.494026  [    0/  451]
Test Error: 
 Accuracy: 64.6%, Avg loss: 0.052355 

Epoch 9
----------------

In [13]:
eval('np.array([1,1,1])').shape

(3,)

In [6]:
from torchsummary import summary

In [10]:
net = BrainNetCNN()
summary(net,input_size=(1,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 32, 64, 1]           2,080
            Conv2d-2            [-1, 32, 1, 64]           2,080
          E2EBlock-3           [-1, 32, 64, 64]               0
            Conv2d-4            [-1, 64, 64, 1]         131,136
            Conv2d-5            [-1, 64, 1, 64]         131,136
          E2EBlock-6           [-1, 64, 64, 64]               0
            Conv2d-7             [-1, 1, 64, 1]           4,097
            Conv2d-8            [-1, 256, 1, 1]          16,640
            Linear-9                  [-1, 128]          32,896
           Linear-10                   [-1, 30]           3,870
           Linear-11                    [-1, 2]              62
Total params: 323,997
Trainable params: 323,997
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/

In [17]:
summary(ukbbSex(),input_size=(40,52))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 256, 1, 52]          10,496
       BatchNorm2d-2           [-1, 256, 1, 52]             512
            Linear-3                   [-1, 64]         852,032
           Dropout-4                   [-1, 64]               0
       BatchNorm1d-5                   [-1, 64]             128
            Linear-6                   [-1, 64]           4,160
           Dropout-7                   [-1, 64]               0
       BatchNorm1d-8                   [-1, 64]             128
            Linear-9                    [-1, 2]             130
          Softmax-10                    [-1, 2]               0
Total params: 867,586
Trainable params: 867,586
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.21
Params size (MB): 3.31
Estimated T

In [44]:
#convolutional network architecture
def model(data, keep_pr):
    #first layer: line-by-line convolution with ReLU and dropout
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.dropout(tf.nn.relu(conv+layer1_biases),keep_pr)

    #second layer: convolution by column with ReLU and dropout
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.dropout(tf.nn.relu(conv+layer2_biases),keep_pr)

    #third layer: fully connected hidden layer with dropout and ReLU
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),keep_pr)

    #fourth (output) layer: fully connected layer with logits as output
    return tf.matmul(hidden, layer4_weights) + layer4_biases

class CCNN(torch.nn.Module):
    def __init__(self,d=64):
        super().__init__()
        self.d = d
        
        self.conv1 = torch.nn.Conv2d(1,64,(1,self.d))
        self.conv2 = torch.nn.Conv2d(64,128,(self.d,1))
        self.fc1 = torch.nn.Linear(128,96)
        self.fc2 = torch.nn.Linear(96,2)

    def forward(self,x):
        x = F.dropout(F.relu(self.conv1(x)),p=0.5)
        x = F.dropout(F.relu(self.conv2(x)),p=0.5)
        x = x.view(x.size(0), -1)
        x = F.dropout(F.relu(self.fc1(x)),p=0.5)
        x = F.dropout(F.relu(self.fc2(x)),p=0.5)
        return x

In [31]:
conv1 = torch.nn.Conv2d(1,64,(1,64)).double()
conv2 = torch.nn.Conv2d(64,128,(64,1)).double()
fc1 = torch.nn.Linear(128,96).double()
fc2 = torch.nn.Linear(96,2).double()

In [74]:
model = CCNN().double()
net = BrainNetCNN().double()

In [80]:
x = next(iter(trainloader))[0]
print(model(x))
print(net(x))

tensor([[0.0000, 0.0051],
        [0.0000, 0.0809],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0169, 0.0000],
        [0.0338, 0.1829],
        [0.0000, 0.0000],
        [0.0423, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0862],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000]], dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([[-0.0142,  0.1391],
        [ 0.0077,  0.1593],
        [-0.0227,  0.1181],
        [-0.0026,  0.1153],
        [ 0.0216,  0.0683],
        [-0.0133,  0.1509],
        [-0.0059,  0.1032],
        [-0.0156,  0.0866],
        [-0.0026,  0.0870],
        [-0.0111,  0.1050],
        [-0.0108,  0.1285],
        [ 0.0139,  0.1089],
        [-0.0100,  0.1118],
        [-0.0021,  0.1296],
        [-0.0197,  0.1298],
        [ 0.0254,  0.1467]], dtype=torch.float64, grad_fn=<LeakyReluBackward0>)
